In [1]:
%load_ext autoreload
%autoreload 2

import os,sys,warnings
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
np.set_printoptions(legacy='1.25')
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 24
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 6
mpl.rcParams['errorbar.capsize'] = 6
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu
yu.flag_fast=False

enss=['b','c','d','e']
enss=['b','c','d']
ens2full={'a24':'cA211.53.24','a':'cA2.09.48','b':'cB211.072.64','c':'cC211.060.80','d':'cD211.054.96','e':'cE211.044.112'}
ens2label={'a24':'A24','a':'A48','b':'B64','c':'C80','d':'D96','e':'E112'}
ens2a={'a24':0.0908,'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692,'e':0.04892} # fm
ens2N={'a24':24,'a':48,'b':64,'c':80,'d':96,'e':112}
ens2N_T={'a24':24*2,'a':48*2,'b':64*2,'c':80*2,'d':96*2,'e':112*2}

# ens2mN={'a':931}
# ens2mpiC={'a':131}
# ens2mpi0={'a':111}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

baseFigPath='fig/analysis_A20/'

def find_t_cloest(ens,t):
    return round(t/ens2a[ens])

path_basedata='/p/project1/ngff/li47/code/projectData/02_discNJN_1D/analysis_A20.h5'

In [2]:
path='data_aux/dat_ignore/analysis_A20.pkl'
with open(path,'rb') as f:
    ens2bare=pickle.load(f)

ens2Njk={}
for ens in enss:
    path=path_basedata
    with h5py.File(path) as f:        
        ens2Njk[ens]=len(f[f'{ens}/mom0/N'])

path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs_me=pickle.load(f)
ens2RCs={ens:{} for ens in enss}
for ens in enss:
    for key in ens2RCs_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs[ens][key]=yu.jackknife_pseudo([ens2RCs_me[ens][key]],np.array([[ens2RCs_me[ens][f'{key}_err']**2+1e-10]]),ens2Njk[ens])[:,0]

In [35]:
# 31 conn@mom0, jg;disc@mom0, jq;disc@mom1_sum

fla2iso={
    'u':[(1/4,'q'),(1/2,'v1'),(1/6,'v2'),(1/12,'v3')],
    'd':[(1/4,'q'),(-1/2,'v1'),(1/6,'v2'),(1/12,'v3')],
    's':[(1/4,'q'),(-1/3,'v2'),(1/12,'v3')],
    'c':[(1/4,'q'),(-1/4,'v3')]
}
fla2iso_conn={
    'u':[(1/2,'q'),(1/2,'v1')],
    'd':[(1/2,'q'),(-1/2,'v1')],
}

ens2rb={ens:{} for ens in enss}
ens2rb['a=#_MA']={}; ens2rb['a=#_const']={}; ens2rb['a=#_linear']={}
stouts=range(0,40+1,2)

for stout in stouts:
    for ens in enss:
        dic_rb=ens2rb[ens]
        dic_b=ens2bare[ens]
        dic_Z=ens2RCs[ens]
        
        for mom in ['mom0','mom1']:
            for method in ['','_sum']:
                dic_b[f'{mom}_jq;disc{method}']=dic_b[f'{mom}_j+;disc{method}']+dic_b[f'{mom}_js;disc{method}']+dic_b[f'{mom}_jc;disc{method}']

        dic_rb[f'jq;{stout}']=dic_Z[f'Zqq^s(mu=nu)']*dic_b[f'mom0_j+;conn'] + dic_Z[f'Zqq^s(mu!=nu)']*dic_b[f'mom1_jq;disc_sum'] + \
            dic_Z[f'Zqg(mu=nu)']*dic_b[f'mom0_jg;stout{stout};disc']
        dic_rb[f'jg;{stout}']=dic_Z[f'Zgq(mu=nu)']*dic_b[f'mom0_j+;conn'] + dic_Z[f'Zgq(mu!=nu)']*dic_b[f'mom1_jq;disc_sum'] + \
            dic_Z[f'Zgg^{stout}(mu=nu)']*dic_b[f'mom0_jg;stout{stout};disc']
            
        dic_rb[f'jv1;{stout}']=dic_Z[f'Zqq(mu=nu)']*dic_b[f'mom0_j-;conn']
        dic_rb[f'jv2;{stout}']=dic_Z[f'Zqq(mu=nu)']*dic_b[f'mom0_j+;conn'] + dic_Z[f'Zqq(mu!=nu)']*(dic_b[f'mom1_j+;disc']-2*dic_b[f'mom1_js;disc'])
        dic_rb[f'jv3;{stout}']=dic_Z[f'Zqq(mu=nu)']*dic_b[f'mom0_j+;conn'] + dic_Z[f'Zqq(mu!=nu)']*(dic_b[f'mom1_j+;disc']+dic_b[f'mom1_js;disc']-3*dic_b[f'mom1_jc;disc'])
            
        dic_rb[f'jq;conn;{stout}']=dic_Z[f'Zqq^s(mu=nu)']*dic_b[f'mom0_j+;conn']
        dic_rb[f'jv1;conn;{stout}']=dic_Z[f'Zqq(mu=nu)']*dic_b[f'mom0_j-;conn']

        dic_rb[f'jtot;{stout}']=dic_rb[f'jq;{stout}']+dic_rb[f'jg;{stout}']
        
        for fla in fla2iso.keys():
            dic_rb[f'j{fla};{stout}']=np.sum([factor*dic_rb[f'j{iso};{stout}'] for factor,iso in fla2iso[fla]],axis=0)
        for fla in fla2iso_conn.keys():
            dic_rb[f'j{fla};conn;{stout}']=np.sum([factor*dic_rb[f'j{iso};conn;{stout}'] for factor,iso in fla2iso_conn[fla]],axis=0)
        
    # continuum extrapolation
    a2s_plt=np.arange(0,0.011,0.001)
    for j in ['jg','jq','jv1','jv2','jv3','jq;conn','jv1;conn']:
        a2s=np.array([ens2a[ens]**2 for ens in enss])
        t=[yu.jackme(ens2rb[ens][f'{j};{stout}']) for ens in enss]
        gs_m=np.array([ele[0] for ele in t])
        gs_e=np.array([ele[1] for ele in t])
        
        fits=[]
        
        # const
        mean=gs_m; err=gs_e
        def fitfunc(a2,g):
            return g+0*a2
        popt,pcov=curve_fit(fitfunc,a2s,mean,sigma=err,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-mean
        color='g'
        chi2R=r.T @ np.linalg.inv(np.diag(err**2)) @ r / (len(enss)-1)

        mean,err,cov=yu.propagateError(lambda pars:fitfunc(a2s_plt,*pars), popt, pcov)
        
        pars_mean=mean; pars_err=err; Ndof=(len(enss)-1); chi2=chi2R*Ndof
        fits.append([pars_mean,pars_err,chi2,Ndof])
        ens2rb['a=#_const'][f'{j};{stout}']=yu.jackknife_pseudo(pars_mean,np.diag(pars_err**2),1000)
            
        # linear
        mean=gs_m; err=gs_e
        def fitfunc(a2,g,c):
            return g+c*a2
        popt,pcov=curve_fit(fitfunc,a2s,mean,sigma=err,absolute_sigma=True)
        r=fitfunc(a2s,*popt)-mean
        color='b'
        chi2R=r.T @ np.linalg.inv(np.diag(err**2)) @ r / (len(enss)-2)

        mean,err,cov=yu.propagateError(lambda pars:fitfunc(a2s_plt,*pars), popt, pcov)
        # ax.plot(a2s2,mean,'--',color=color)
        # ax.fill_between(a2s2,mean-err,mean+err,color=color,alpha=0.2)
        # ax.errorbar([0],[popt[0]],np.sqrt(pcov[0,0]),color=color, label=yu.un2str(popt[0],np.sqrt(pcov[0,0])))
        # ylim=ax.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        # ax.annotate("%0.1f" %chi2R,(0,popt[0]-np.sqrt(pcov[0,0])-chi2_shift),color=color)
        
        pars_mean=mean; pars_err=err; Ndof=(len(enss)-2); chi2=chi2R*Ndof
        fits.append([pars_mean,pars_err,chi2,Ndof])
        ens2rb['a=#_linear'][f'{j};{stout}']=yu.jackknife_pseudo(pars_mean,np.diag(pars_err**2),1000)
        
        # model average
        (pars_mean_MA,pars_err_MA,props)=yu.modelAvg(fits)
        ens2rb['a=#_MA'][f'{j};{stout}']=yu.jackknife_pseudo(pars_mean_MA,np.diag(pars_err_MA**2),1000)
    
    for ft in ['const','linear','MA']:
        ens2rb[f'a=#_{ft}'][f'jtot;{stout}']=ens2rb[f'a=#_{ft}'][f'jq;{stout}']+ens2rb[f'a=#_{ft}'][f'jg;{stout}']
    
        for fla in fla2iso.keys():
            ens2rb[f'a=#_{ft}'][f'j{fla};{stout}']=np.sum([factor*ens2rb[f'a=#_{ft}'][f'j{iso};{stout}'] for factor,iso in fla2iso[fla]],axis=0)
        for fla in fla2iso_conn.keys():
            ens2rb[f'a=#_{ft}'][f'j{fla};conn;{stout}']=np.sum([factor*ens2rb[f'a=#_{ft}'][f'j{iso};conn;{stout}'] for factor,iso in fla2iso_conn[fla]],axis=0)

In [ ]:
fig, axs = yu.getFigAxs(3,1,Lrow=4,Lcol=12)

js=['jtot','jg','jq']
for ij,j in enumerate(js):
    ax=axs[ij,0]
    ax.set_ylim({'jtot':[0.5,1.5],'jg':[0,1],'jq':[0,1]}[j])
    ax.set_ylabel(j)
    if j=='jtot':
        ax.axhline(1,ls='--')
    
    colors=['r','g','b']
    for stout in stouts:
        if stout%2!=0:
            continue
        for iens,ens in enumerate(enss):
            plt_x=stout+0.2*iens; plt_y,plt_yerr=yu.jackme(ens2rb[ens][f'{j};{stout}'])
            ax.errorbar(plt_x,plt_y,plt_yerr,color=colors[iens],label=ens if stout==stouts[0] else None)

        plt_x=stout-0.2; plt_y,plt_yerr=yu.jackme(ens2rb['a=#_MA'][f'{j};{stout}'][:,0])
        ax.errorbar(plt_x,plt_y,plt_yerr,color='orange',label='c.l._MA' if stout==stouts[0] else None)
            
        if stout==10:
            print(j,'MA',plt_y,plt_yerr)
        
        plt_x=stout-0.4; plt_y,plt_yerr=yu.jackme(ens2rb['a=#_const'][f'{j};{stout}'][:,0])
        ax.errorbar(plt_x,plt_y,plt_yerr,color='purple',label='c.l._const' if stout==stouts[0] else None)
        if stout==10:
            print(j,'const',plt_y,plt_yerr)
        plt_x=stout-0.6; plt_y,plt_yerr=yu.jackme(ens2rb['a=#_linear'][f'{j};{stout}'][:,0])
        ax.errorbar(plt_x,plt_y,plt_yerr,color='brown',label='c.l._linear' if stout==stouts[0] else None)
        if stout==10:
            print(j,'linear',plt_y,plt_yerr)

    if j=='jtot':
        ax.legend(fontsize=10)
print()
plt.tight_layout()
os.makedirs(f'{baseFigPath}/final1',exist_ok=True)
# plt.savefig(f'{baseFigPath}/final1/stoutDependence.pdf')
plt.close()

jtot MA 1.009629849262865 0.1978307534968782
jtot const 0.9541083252967093 0.06339168550893697
jtot linear 1.128532164608951 0.3027158024115567
jg MA 0.5291585031105057 0.16445462829091784
jg const 0.4753128583519246 0.050445320802727754
jg linear 0.6435079637957319 0.2446284201391701
jq MA 0.4804713461523593 0.10208963475086406
jq const 0.47879546694478464 0.03833703088501616
jq linear 0.48502420081321895 0.18632051910005828



In [98]:
stout=10

j2color={'jq':'purple','jg':'cyan','jtot':'grey','ju':'r','jd':'g','js':'b','jc':'orange'}
j2label={'jq':'q','jg':'g','jtot':'tot','ju':'u','jd':'d','js':'s','jc':'c'}

fig, axs = yu.getFigAxs(2,1,Lrow=4,Lcol=6,sharex=True)
ax=axs[0,0]
ax.set_xlim([0,0.01])
ax.set_xticks([0,0.003,0.006,0.009])
ax.set_ylim([0.2,1.4])
ax.set_ylabel(r'$\langle x\rangle$')
ax.axhline(1,color='black',ls='--',marker='')
# ax.axvline(0,color='black',ls='dotted',marker='')
js=['jtot','jq','jg']
for ij,j in enumerate(js):
    color=j2color[j]
    for iens,ens in enumerate(enss):
        plt_x=ens2a[ens]**2+(ij-len(js)/2)*5e-5; plt_y,plt_yerr=yu.jackme(ens2rb[ens][f'{j};{stout}'])
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color)

    mean,err=yu.jackme(ens2rb['a=#_MA'][f'{j};{stout}'])
    x=a2s_plt; ymin=mean-err; ymax=mean+err
    ax.plot(x,mean,color=color,linestyle='--',marker='')
    ax.fill_between(x, ymin, ymax, color=color, alpha=0.1, label=rf'$\langle x\rangle _{{{j2label[j]}}}=$'+yu.un2str(mean[0],err[0]))
ax.legend(fontsize=10)

ax=axs[1,0]
ax.set_xlabel(r'$a^2$ [fm$^2$]')
ax.set_ylim([-0.1,0.55])
ax.set_ylabel(r'$\langle x\rangle$')
ax.axhline(0,color='black',ls='--',marker='')
js=['ju','jd','js','jc']
for ij,j in enumerate(js):
    color=j2color[j]
    for iens,ens in enumerate(enss):
        plt_x=ens2a[ens]**2+(ij-len(js)/2)*5e-5; plt_y,plt_yerr=yu.jackme(ens2rb[ens][f'{j};{stout}'])
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color)

    mean,err=yu.jackme(ens2rb['a=#_MA'][f'{j};{stout}'])
    x=a2s_plt; ymin=mean-err; ymax=mean+err
    ax.plot(x,mean,color=color,linestyle='--',marker='')
    ax.fill_between(x, ymin, ymax, color=color, alpha=0.1, label=rf'$\langle x\rangle _{{{j2label[j]}}}=$'+yu.un2str(mean[0],err[0],forceResult=1))
ax.legend(fontsize=10)

plt.tight_layout()
# plt.savefig(f'{baseFigPath}/final1/a2Dependence.pdf',bbox_inches="tight")
plt.close()

In [90]:
stout=10

def get(j,stout,part):
    if part=='full':
        return ens2rb['a=#_MA'][f'{j};{stout}'][:,0]
    if part=='conn':
        if j not in ['ju','jd','jq']:
            return ens2rb['a=#_MA'][f'{j};{stout}'][:,0]*0
        return ens2rb['a=#_MA'][f'{j};conn;{stout}'][:,0]

fig, axs = yu.getFigAxs(1,1,Lrow=6,Lcol=8)

js = ['ju','jd','js','jc','jq','jg','jtot']
# labels = [r'$u^+$', r'$d^+$', r'$s^+$', r'$c^+$', r'$\sum_{q^+ = u,d,s,c}$', r'$g$', 'Total']
labels = [r'$u$', r'$d$', r'$s$', r'$c$', r'$q$', r'$g$', 'tot']
colors = ['r','g','b','orange','purple','cyan','grey']

ax=axs[0,0]
ax.axhline(1,color='black',ls='--',marker='')
ax.axhline(0,color='black',ls='--',marker='')
ax.set_ylabel(r'$\langle x\rangle$')

x = np.arange(len(labels))
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=12)
ax.set_xlim([x[0]-3/4,x[-1]+3/4])

mes_conn=[yu.jackme(get(j,stout,'conn')) for j in js]
mes=[yu.jackme(get(j,stout,'full')) for j in js]

factor=1
    
ax.bar(x, [ele[0] for ele in mes_conn], capsize=5, color=colors, alpha=0.8, width=0.3, edgecolor='grey')
bars=ax.bar(x, [ele[0] for ele in mes], yerr=[ele[1] for ele in mes], capsize=5, color=colors, alpha=0.2, width=0.5, edgecolor='grey')

func=lambda res:'{:.1f}({:.1f})%'.format(res[0]*100/factor,res[1]*100/factor)
percentages=[func(ele) for ele in mes]

for i, (bar, pct) in enumerate(zip(bars, percentages)):
    height = bar.get_height()
    height = 0.01 if height < 0.2 else height - 0.2
    ax.text(bar.get_x() - 0.1, height, pct,
            ha='center', va='bottom', fontsize=10, rotation=90)
        
plt.tight_layout()
# plt.savefig(f'{baseFigPath}/final1/percentages.pdf',bbox_inches="tight")
plt.close()